<a href="https://colab.research.google.com/github/xuanyuw/NOVA1_variation_LFP/blob/main/analyze_LFP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/
